In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sklearn.preprocessing
import matplotlib.pyplot as plt
import datetime
import os
import math
from numpy import linalg as LA
from google.colab import drive
from bokeh.plotting import figure, show, output_notebook, output_file, save
from bokeh.models import LinearAxis, Range1d, ColumnDataSource, HoverTool
from bokeh.embed import file_html
from bokeh.layouts import row, gridplot
from bokeh.palettes import Spectral6 as palette
from bokeh.io import curdoc
import itertools  
!pip install xlsxwriter
import xlsxwriter
drive.mount('/content/drive')

output_notebook()

     |████████████████████████████████| 148 kB 5.3 MB/s 
Mounted at /content/drive


In [ ]:
# folder_path 해당 폴더 안 모든 파일을 file_list에 저장
folder_path = "/content/drive/MyDrive/Robostar/W8"
file_list = os.listdir(folder_path)

# ".csv" 파일만 file_list_py에 저장
# ".csv" 파일 제외한 모든 파일 삭제
file_list_py = []
for file in file_list:
  if file.endswith(".csv"):
    file_list_py.append(file)
  else:
    os.remove(folder_path + "/" + file)
file_list_py = sorted(file_list_py)

print(file_list)
print(file_list_py)

print("check the file remove: ")
print(os.listdir(folder_path))

['20210730_170407_MONITORING.csv', '20210730_170431_MONITORING.csv', '20210730_170441_MONITORING.csv', '20210730_170448_MONITORING.csv', '20210730_170455_MONITORING.csv', 'Base-FKBase_xy.html', 'Base-FKBase_xz.html', 'Base-FKBase_yz.html', 'Err 4-5.html', 'Err 4-6.html', 'Err 5-6.html', 'norm.html', 'SVD_Manip_Velocity4.html', 'joint_vel1-6.html']
['20210730_170407_MONITORING.csv', '20210730_170431_MONITORING.csv', '20210730_170441_MONITORING.csv', '20210730_170448_MONITORING.csv', '20210730_170455_MONITORING.csv']
check the file remove: 
['20210730_170407_MONITORING.csv', '20210730_170431_MONITORING.csv', '20210730_170441_MONITORING.csv', '20210730_170448_MONITORING.csv', '20210730_170455_MONITORING.csv']


In [ ]:
# 각 파일 raw data 불러오기 , ","를 기준으로 Data 분할
#첫 행을 column으로 Data 분할

raw_log_datas = []
for i in file_list_py:
  raw_log_datas.append(pd.read_csv(folder_path + '/' + i, sep=",", encoding="cp949"))

raw_log_datas

[          date          time  robot_id  ...  after_xpos4  after_xpos5  after_xpos6
 0     21-07-30  17:04:12.283         1  ...    -179.0797      13.8050      -0.9204
 1     21-07-30  17:04:12.285         1  ...    -179.0797      13.8050      -0.9204
 2     21-07-30  17:04:12.287         1  ...    -179.0797      13.8050      -0.9204
 3     21-07-30  17:04:12.289         1  ...    -179.0797      13.8050      -0.9204
 4     21-07-30  17:04:12.291         1  ...    -179.0797      13.8050      -0.9204
 ...        ...           ...       ...  ...          ...          ...          ...
 7341  21-07-30  17:04:26.965         1  ...    -179.0798      13.8036      -0.9234
 7342  21-07-30  17:04:26.967         1  ...    -179.0798      13.8035      -0.9230
 7343  21-07-30  17:04:26.969         1  ...    -179.0798      13.8031      -0.9227
 7344  21-07-30  17:04:26.971         1  ...    -179.0798      13.8033      -0.9214
 7345  21-07-30  17:04:26.973         1  ...    -179.0797      13.8051      

In [ ]:
log_datas = raw_log_datas

# Graph 1 : SVD, Manip, Velocity4 (RefJoint4)

x_axis_name, y_axis_name = "Ref_Joint5", "SVD, Manip, Velocity4"
plots = []
for i in range(0, len(file_list_py)):
  curdoc().theme = "caliber"
  TOOLS = "pan,wheel_zoom,box_select,lasso_select,reset, save, hover"
  TOOLTIPS = [
    ("index", "$x"),
    ("y_value", "$y)"),
  ]

  svd_color, manip_color, velocity4_color = palette[0], palette[3], palette[5]

  p = figure(title = file_list_py[i], x_axis_label=x_axis_name, toolbar_location="right", plot_width=1200, plot_height=600, tools=TOOLS, tooltips=TOOLTIPS)
  p.extra_y_ranges = {"SVD" :Range1d(start=min(log_datas[i]["singular_value"]), end=max(log_datas[i]["singular_value"]))}
  p.circle(log_datas[i]["joint_pos_ref5"], log_datas[i]["singular_value"], y_range_name="SVD", line_color=svd_color, legend_label="SVD", size=3, alpha=0.4)
  p.add_layout(LinearAxis(y_range_name="SVD", axis_label="SVD", axis_label_text_color=svd_color, axis_line_color=svd_color), "left")

  p.extra_y_ranges["Manip"] = Range1d(start=min(log_datas[i]["manipulability"]), end=max(log_datas[i]["manipulability"]))
  p.circle(log_datas[i]["joint_pos_ref5"], log_datas[i]["manipulability"], y_range_name="Manip", line_color=manip_color, legend_label="Manip", size=3, alpha=0.4)
  p.add_layout(LinearAxis(y_range_name="Manip", axis_label="Manip", axis_label_text_color=manip_color, axis_line_color=manip_color), "right")

  p.extra_y_ranges["Velocity4"] = Range1d(start=min(log_datas[i]["joint_vel4"]), end=max(log_datas[i]["joint_vel4"]))
  p.circle(log_datas[i]["joint_pos_ref5"], log_datas[i]["joint_vel4"], y_range_name="Velocity4", line_color=velocity4_color, legend_label="Velocity4", size=3, alpha=0.4) 
  p.add_layout(LinearAxis(y_range_name="Velocity4", axis_label="Velocity4", axis_label_text_color=velocity4_color, axis_line_color=velocity4_color), "right")

  p.legend.location = "top_right"
  p.legend.click_policy = "hide"
  p.background_fill_color="#f0f0f0"
  p.grid.grid_line_color="#f5f5f5"
  
  
 
  
  plots.append(p)
  # show, svae 같이 사용시 오류 발생
  # 같이 사용하고 싶다면 figure 설정을 2번 해야 하는 것 같음
  #show(p)

plot = gridplot(plots, ncols=1, toolbar_location="right", plot_width=1200, plot_height=600)
output_file(folder_path + "/" + "SVD_Manip_Velocity4" + '.html')
#save(plot)
show(plot)



In [52]:
# Graph 2 : joint_vel1 ~ joint_vel6
# x축을 위한 index 열 생성
for i in range(0, len(file_list_py)):
  index = []
  for j in range(0, len(log_datas[i])):
    index.append(j)
  log_datas[i]["index"] = index

plots = []
x_axis_name, y_axis_name = "index", ""
colors = itertools.cycle(palette)

for i in range(0, len(file_list_py)):
  curdoc().theme = 'caliber'

  TOOLS = "pan,wheel_zoom,box_select,lasso_select,reset, save"
  TOOLTIPS = [
    ("index", "$x"),
    ("y_value", "$y)"),
  ]
  p = figure(title = file_list_py[i], x_axis_label=x_axis_name, toolbar_location="right", plot_width=1200, plot_height=600, tools=TOOLS, tooltips=TOOLTIPS)  
  for x, color in zip(range(6), colors):
    column_name = "joint_vel"
    column_name += str(x + 1)
    p.circle(log_datas[i]["index"], log_datas[i][column_name], legend_label=column_name, color=color, alpha=0.4, size=3)

  p.legend.location = "top_left"
  p.legend.click_policy = "hide"
  p.background_fill_color="#f0f0f0"
  p.grid.grid_line_color="#f5f5f5"

  print(file_list_py[i])
  plots.append(p)

plot = gridplot(plots, ncols=1, toolbar_location="right", plot_width=1200, plot_height=600)
output_file(folder_path + "/" + "joint_vel1-6" + '.html')
show(plot)
#save(plot)

20210730_170407_MONITORING.csv
20210730_170431_MONITORING.csv
20210730_170441_MONITORING.csv
20210730_170448_MONITORING.csv
20210730_170455_MONITORING.csv


In [ ]:
# 특이점 회피 알고리즘 에러 계산 코드

# 함수
def rotx(t) :
    res = np.zeros((3,3))
    ct = np.cos(t)
    st = np.sin(t)

    res[0][0] = 1  
    res[0][1] = 0  
    res[0][2] = 0

    res[1][0] = 0
    res[1][1] = ct  
    res[1][2] = -1.0 * st

    res[2][0] = 0  
    res[2][1] = st  
    res[2][2] = ct
    
    return res

def roty(t) :
    res = np.zeros((3,3))
    ct = np.cos(t)
    st = np.sin(t)

    res[0][0] = ct
    res[0][1] = 0  
    res[0][2] = st

    res[1][0] = 0
    res[1][1] = 1  
    res[1][2] = 0

    res[2][0] = -1.0 * st  
    res[2][1] = 0
    res[2][2] = ct
    
    return res

def rotz(t) :
    res = np.zeros((3,3))
    ct = np.cos(t)
    st = np.sin(t)

    res[0][0] = ct
    res[0][1] = -1.0 * st  
    res[0][2] = 0

    res[1][0] = st
    res[1][1] = ct  
    res[1][2] = 0

    res[2][0] = 0
    res[2][1] = 0
    res[2][2] = 1
    
    return res

def eulXYZ2tr(phi, theta, psi) :
    ret_matrix = np.zeros((3,3))
    temp_matrix = np.zeros((3,3))

    r_phi = phi * math.pi/180
    r_theta = theta * math.pi/180
    r_psi = psi * math.pi/180

    rotx_matrix = rotx(r_phi)
    roty_matrix = roty(r_theta)
    rotz_matrix = rotz(r_psi)

    ret_matrix = (rotx_matrix @ roty_matrix) @ rotz_matrix

    return ret_matrix

def pose2tr(pos):
    ret_matrix = np.zeros((4,4))
    rot_matrix = np.zeros((3,3))

    rot_matrix = eulXYZ2tr(pos[3], pos[4], pos[5])

    ret_matrix[0][0] = rot_matrix[0][0]
    ret_matrix[0][1] = rot_matrix[0][1]
    ret_matrix[0][2] = rot_matrix[0][2]
    ret_matrix[0][3] = pos[0]

    ret_matrix[1][0] = rot_matrix[1][0]
    ret_matrix[1][1] = rot_matrix[1][1]
    ret_matrix[1][2] = rot_matrix[1][2]
    ret_matrix[1][3] = pos[1]

    ret_matrix[2][0] = rot_matrix[2][0]
    ret_matrix[2][1] = rot_matrix[2][1]
    ret_matrix[2][2] = rot_matrix[2][2]    
    ret_matrix[2][3] = pos[2]

    ret_matrix[3][0] = 0.0
    ret_matrix[3][1] = 0.0
    ret_matrix[3][2] = 0.0
    ret_matrix[3][3] = 1.0

    return ret_matrix

def tr2delta(T0, T1) :
    ret_matrix = np.zeros((6,1))
    temp_matrix = np.zeros((4,4))
    Tinv = np.zeros((4,4))
    for i in (0, 3) :
        for j in (0, 3) :
            Tinv[i][j] = T0[j][i]

            Tinv[0][3] = -1.0 * (Tinv[0][0] * T0[0][3] + Tinv[0][1] * T0[1][3] + Tinv[0][2] * T0[2][3])
            Tinv[1][3] = -1.0 * (Tinv[1][0] * T0[0][3] + Tinv[1][1] * T0[1][3] + Tinv[1][2] * T0[2][3])
            Tinv[2][3] = -1.0 * (Tinv[2][0] * T0[0][3] + Tinv[2][1] * T0[1][3] + Tinv[2][2] * T0[2][3])

            Tinv[3][3] = 1.0

    temp_matrix = Tinv @ T1
    
    #print (Tinv)
    #print (temp_matrix)

    ret_matrix[0][0] = temp_matrix[0][3]
    ret_matrix[1][0] = temp_matrix[1][3]
    ret_matrix[2][0] = temp_matrix[2][3]

    ret_matrix[3][0] = 0.5 * (temp_matrix[2][1]-temp_matrix[1][2])
    ret_matrix[4][0] = 0.5 * (temp_matrix[0][2]-temp_matrix[2][0])
    ret_matrix[5][0] = 0.5 * (temp_matrix[1][0]-temp_matrix[0][1])

    return ret_matrix



In [ ]:
# RefBaseMatrix 구현 코드 작성

# step1 creat RefBase transform matrix
#RefBaseMatrix = np.zeros((4,4))
#RefBase = [379.99833, -10.39186, 637.10311, -173.18176, 89.97753, -6.81795]
#RefBaseMatrix = pose2tr(RefBase)

#base_pos_ref1 ~ base_pos_ref6
for i in range(0, len(file_list_py)):
  arr_RefBaseMatirx = []
  for j in range(0, len(log_datas[i])):
    RefBaseMatrix = np.zeros((4,4))
    RefBase = [log_datas[i]["base_pos_ref1"][j], log_datas[i]["base_pos_ref2"][j], log_datas[i]["base_pos_ref3"][j], log_datas[i]["base_pos_ref4"][j], log_datas[i]["base_pos_ref5"][j], log_datas[i]["base_pos_ref6"][j]]
    RefBaseMatrix = pose2tr(RefBase)
    arr_RefBaseMatirx.append(RefBaseMatrix)
  log_datas[i]["RefBaseMatrix"] = arr_RefBaseMatirx




In [ ]:
# RefBaseMatrix 생성 확인

for i in range(0, len(file_list_py)):
  print(log_datas[i].head())

       date  ...                                      RefBaseMatrix
0  21-07-30  ...  [[0.9709881638405319, 0.015599305832557775, 0....
1  21-07-30  ...  [[0.9709881638405319, 0.015599305832557775, 0....
2  21-07-30  ...  [[0.9709881638405319, 0.015599305832557775, 0....
3  21-07-30  ...  [[0.9709881638405319, 0.015599305832557775, 0....
4  21-07-30  ...  [[0.9709881638405319, 0.015599305832557775, 0....

[5 rows x 44 columns]
       date  ...                                      RefBaseMatrix
0  21-07-30  ...  [[0.9709881638405319, 0.015599305832557775, 0....
1  21-07-30  ...  [[0.9709881638405319, 0.015599305832557775, 0....
2  21-07-30  ...  [[0.9709881638405319, 0.015599305832557775, 0....
3  21-07-30  ...  [[0.9709881638405319, 0.015599305832557775, 0....
4  21-07-30  ...  [[0.9709881638405319, 0.015599305832557775, 0....

[5 rows x 44 columns]
       date  ...                                      RefBaseMatrix
0  21-07-30  ...  [[0.9709881638405319, 0.015599305832557775, 0....
1 

In [ ]:
# FKBase 구현 코드 작성

# step2 creat FKBase transform matrix
#FKBaseMatrix = np.zeros((4,4))
#FKBase = [379.99731, -10.38402, 637.09923, 58.848, 87.59157, 121.17104] # todo
#FKBaseMatrix = pose2tr(FKBase)

#after_xpos1 ~ after_xpos6
for i in range(0, len(file_list_py)):
  arr_FKBaseMatirx = []
  for j in range(0, len(log_datas[i])):
    FKBaseMatrix = np.zeros((4,4))
    FKBase = [log_datas[i]["after_xpos1"][j], log_datas[i]["after_xpos2"][j], log_datas[i]["after_xpos3"][j], log_datas[i]["after_xpos4"][j], log_datas[i]["after_xpos5"][j], log_datas[i]["after_xpos6"][j]]
    FKBaseMatrix = pose2tr(FKBase)
    arr_FKBaseMatirx.append(FKBaseMatrix)
  log_datas[i]["FKBaseMatrix"] = arr_FKBaseMatirx



In [ ]:
# FKBaseMatrix 생성 확인

for i in range(0, len(file_list_py)):
  print(log_datas[i].head())


       date  ...                                       FKBaseMatrix
0  21-07-30  ...  [[0.9709881638405319, 0.015599305832557775, 0....
1  21-07-30  ...  [[0.9709881638405319, 0.015599305832557775, 0....
2  21-07-30  ...  [[0.9709881638405319, 0.015599305832557775, 0....
3  21-07-30  ...  [[0.9709881638405319, 0.015599305832557775, 0....
4  21-07-30  ...  [[0.9709881638405319, 0.015599305832557775, 0....

[5 rows x 45 columns]
       date  ...                                       FKBaseMatrix
0  21-07-30  ...  [[0.9709881638405319, 0.015599305832557775, 0....
1  21-07-30  ...  [[0.9709881638405319, 0.015599305832557775, 0....
2  21-07-30  ...  [[0.9709881638405319, 0.015599305832557775, 0....
3  21-07-30  ...  [[0.9709881638405319, 0.015599305832557775, 0....
4  21-07-30  ...  [[0.9709881638405319, 0.015599305832557775, 0....

[5 rows x 45 columns]
       date  ...                                       FKBaseMatrix
0  21-07-30  ...  [[0.9709881638405319, 0.015599305832557775, 0....
1 

In [ ]:
# step 3 create error vector

#Err_vector = np.zeros((6,1))
#Err_vector = tr2delta(RefBaseMatrix, FKBaseMatrix)
#norm = LA.norm(Err_vector)


for i in range(0, len(file_list_py)):
  arr_Err_vector = []
  arr_norm = []
  for j in range(0, len(log_datas[i])):
    Err_vector = np.zeros((6,1))
    Err_vector = tr2delta(log_datas[i]["RefBaseMatrix"][j], log_datas[i]["FKBaseMatrix"][j])
    arr_Err_vector.append(Err_vector)
    arr_norm.append(LA.norm(Err_vector))
  log_datas[i]["Err_vector"] = arr_Err_vector
  log_datas[i]["norm"] = arr_norm

In [ ]:
log_datas[0].head()

,date,time,robot_id,heartbeat,section_block,rstate,alarm_cnt,tool,user,data_cnt,monitor_idx2,joint_vel1,joint_vel2,joint_vel3,joint_vel4,joint_vel5,joint_vel6,monitor_idx3,joint_pos_ref1,joint_pos_ref2,joint_pos_ref3,joint_pos_ref4,joint_pos_ref5,joint_pos_ref6,monitor_idx10,base_pos_ref1,base_pos_ref2,base_pos_ref3,base_pos_ref4,base_pos_ref5,base_pos_ref6,monitor_idx19,manipulability,singular_value,lambda,monitor_idx51,after_xpos1,after_xpos2,after_xpos3,after_xpos4,after_xpos5,after_xpos6,index,RefBaseMatrix,FKBaseMatrix,Err_vector,norm
0,21-07-30,17:04:12.283,1,0,1,16550,0,0,0,5,2,-0.0118,0.0197,0.0260,-0.0255,-0.0111,-0.0192,3,3.74,40.002,36.163,3.851,0.0,-0.92,10,323.658,21.1569,208.1476,-179.0797,13.805,-0.9204,19,0.0,0.0,10.0,51,323.658,21.1569,208.1476,-179.0797,13.805,-0.9204,0,"[[0.9709881638405319, 0.015599305832557775, 0....","[[0.9709881638405319, 0.015599305832557775, 0....","[[0.0], [0.0], [0.0], [0.0], [0.11584772594041...",0.116095
1,21-07-30,17:04:12.285,1,1,1,174,0,0,0,5,2,-0.0115,0.0465,-0.0407,-0.0463,-0.0016,-0.0108,3,3.74,40.002,36.163,3.851,0.0,-0.92,10,323.658,21.1569,208.1476,-179.0797,13.805,-0.9204,19,0.0,0.0,10.0,51,323.658,21.1569,208.1476,-179.0797,13.805,-0.9204,1,"[[0.9709881638405319, 0.015599305832557775, 0....","[[0.9709881638405319, 0.015599305832557775, 0....","[[0.0], [0.0], [0.0], [0.0], [0.11584772594041...",0.116095
2,21-07-30,17:04:12.287,1,0,1,174,0,0,0,5,2,0.0013,-0.0208,-0.0321,0.0177,0.0020,0.0057,3,3.74,40.002,36.163,3.851,0.0,-0.92,10,323.658,21.1569,208.1476,-179.0797,13.805,-0.9204,19,0.0,0.0,10.0,51,323.658,21.1569,208.1476,-179.0797,13.805,-0.9204,2,"[[0.9709881638405319, 0.015599305832557775, 0....","[[0.9709881638405319, 0.015599305832557775, 0....","[[0.0], [0.0], [0.0], [0.0], [0.11584772594041...",0.116095
3,21-07-30,17:04:12.289,1,1,1,174,0,0,0,5,2,0.0121,-0.0448,0.0338,0.0438,0.0085,0.0054,3,3.74,40.002,36.163,3.851,0.0,-0.92,10,323.658,21.1569,208.1476,-179.0797,13.805,-0.9204,19,0.0,0.0,10.0,51,323.658,21.1569,208.1476,-179.0797,13.805,-0.9204,3,"[[0.9709881638405319, 0.015599305832557775, 0....","[[0.9709881638405319, 0.015599305832557775, 0....","[[0.0], [0.0], [0.0], [0.0], [0.11584772594041...",0.116095
4,21-07-30,17:04:12.291,1,0,1,174,0,0,0,5,2,-0.0064,0.0069,0.0370,-0.0124,-0.0059,-0.0099,3,3.74,40.002,36.163,3.851,0.0,-0.92,10,323.658,21.1569,208.1476,-179.0797,13.805,-0.9204,19,0.0,0.0,10.0,51,323.658,21.1569,208.1476,-179.0797,13.805,-0.9204,4,"[[0.9709881638405319, 0.015599305832557775, 0....","[[0.9709881638405319, 0.015599305832557775, 0....","[[0.0], [0.0], [0.0], [0.0], [0.11584772594041...",0.116095


In [ ]:
norm_max, norm_min = log_datas[0]["norm"][0], log_datas[0]["norm"][0] 
for i in range(0, len(file_list_py)):
  for j in range(0, len(log_datas[i])):
    if norm_max < log_datas[i]["norm"][j]:
      norm_max = log_datas[i]["norm"][j]
    if norm_min > log_datas[i]["norm"][j]:
      norm_min = log_datas[i]["norm"][j]


x_axis_name, y_axis_name = "index", ""
for i in range(0, len(file_list_py)):
  curdoc().theme = "caliber"

  TOOLS = "pan,wheel_zoom,box_select,lasso_select,reset, save"
  TOOLTIPS = [
    ("index", "$x"),
    ("y_value", "$y)"),
  ]

  p = figure(title = file_list_py[i], x_axis_label=x_axis_name, toolbar_location="below", plot_width=1200, plot_height=900, tools=TOOLS, tooltips=TOOLTIPS)  
  p.extra_y_ranges = {"norm" : Range1d(start=norm_min, end=norm_max)}
  p.circle(log_datas[i]["index"], log_datas[i]["norm"], y_range_name="norm", legend_label="norm", color=palette[0], size=3, alpha=0.4)
  p.add_layout(LinearAxis(y_range_name="norm", axis_label="norm", axis_label_text_color=palette[0], axis_line_color=palette[0]), "right")

  p.extra_y_ranges["joint_vel4"] =  Range1d(start=min(log_datas[i]["joint_vel4"]), end=max(log_datas[i]["joint_vel4"]))
  p.circle(log_datas[i]["index"], log_datas[i]["joint_vel4"], y_range_name="joint_vel4", legend_label="joint_vel4", color=palette[5], size=3, alpha=0.4) 
  p.add_layout(LinearAxis(y_range_name="joint_vel4", axis_label="joint_vel4", axis_label_text_color=palette[5], axis_line_color=palette[5]), "left")

  p.background_fill_color="#f0f0f0"
  p.grid.grid_line_color="#f5f5f5"

  show(p)


In [ ]:
print(log_datas[0]["Err_vector"][0])

[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.11584773]
 [-0.00757337]]


In [ ]:
# Error_x,y,z = 각 RefBase x,y,z - 각 FKBase x,y,z  
# Err_vector 4,5,6축 rad를 degree로 변환


for i in range(0, len(file_list_py)):
  log_datas[i]["Error_x"] = log_datas[i]["base_pos_ref4"] - log_datas[i]["after_xpos4"]
  log_datas[i]["Error_y"] = log_datas[i]["base_pos_ref5"] - log_datas[i]["after_xpos5"]
  log_datas[i]["Error_z"] = log_datas[i]["base_pos_ref6"] - log_datas[i]["after_xpos6"]

  for x in range(0, 3):
    # Err_vector 4축 -> 5축 -> 6축
    log_datas[0]["Err_vector"][x+3] *= 180

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [ ]:
# Error_x,y,z / Err_vector 확인

print(log_datas[0]["Error_x"])
print(log_datas[0]["Error_y"])
print(log_datas[0]["Error_z"])
print(log_datas[0]["Err_vector"])


0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
7341    0.0
7342    0.0
7343    0.0
7344    0.0
7345    0.0
Name: Error_x, Length: 7346, dtype: float64
0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
7341    0.0
7342    0.0
7343    0.0
7344    0.0
7345    0.0
Name: Error_y, Length: 7346, dtype: float64
0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
7341    0.0
7342    0.0
7343    0.0
7344    0.0
7345    0.0
Name: Error_z, Length: 7346, dtype: float64
0       [[0.0], [0.0], [0.0], [0.0], [0.11584772594041...
1       [[0.0], [0.0], [0.0], [0.0], [0.11584772594041...
2       [[0.0], [0.0], [0.0], [0.0], [0.11584772594041...
3       [[0.0], [0.0], [0.0], [0.0], [21890215580.9772...
4       [[0.0], [0.0], [0.0], [0.0], [21890215580.9772...
                              ...                        
7341    [[0.0], [0.0], [0.0], [0.0], [0.11583680356904...
7342    [[0.0], [0.0], [0.0], [0.0], [0.11583604339248...
7343    

In [45]:
# x_y Graph
plots = []
for i in range(0, len(file_list_py)):
  curdoc().theme = "caliber"

  TOOLTIPS = [
    ("x_value", "$x"),
    ("y_value", "$y)"),
  ]
  p = figure(title = file_list_py[i], x_axis_label="Error_x", y_axis_label="Error_y", toolbar_location="below", plot_width=1200, plot_height=600, tooltips=TOOLTIPS)
  p.circle(log_datas[i]["Error_x"], log_datas[i]["Error_y"], legend_label="x_y", color=palette[0], size=3, alpha=0.4)
  p.legend.location = "top_left"
  p.legend.click_policy = "hide"
  p.background_fill_color="#f0f0f0"
  p.grid.grid_line_color="#f5f5f5"
  plots.append(p)

plot = gridplot(plots, ncols=1, toolbar_location="right", plot_width=1200, plot_height=600)
output_file(folder_path + "/Base-FKBase_xy.html")
show(plot)
#save(plot)

# x_z Graph
plots = []
for i in range(0, len(file_list_py)):
  curdoc().theme = "caliber"
  
  TOOLTIPS = [
    ("x_value", "$x"),
    ("z_value", "$y)"),
  ]
  p = figure(title = file_list_py[i], x_axis_label="Error_x", y_axis_label="Error_z", toolbar_location="below", plot_width=1200, plot_height=600, tooltips=TOOLTIPS)
  p.circle(log_datas[i]["Error_x"], log_datas[i]["Error_z"], legend_label="x_z", color=palette[2], size=3, alpha=0.4)
  p.legend.location = "top_left"
  p.legend.click_policy = "hide"
  p.background_fill_color="#f0f0f0"
  p.grid.grid_line_color="#f5f5f5"
  plots.append(p)

plot = gridplot(plots, ncols=1, toolbar_location="right", plot_width=1200, plot_height=600)
output_file(folder_path + "/Base-FKBase_xz.html")
show(plot)
#save(plot)

#y_z Graph
plots = []
for i in range(0, len(file_list_py)):
  curdoc().theme = "caliber"
  
  TOOLTIPS = [
    ("y_value", "$x"),
    ("z_value", "$y)"),
  ]
  p = figure(title = file_list_py[i], x_axis_label="Error_y", y_axis_label="Error_z", toolbar_location="below", plot_width=1200, plot_height=600, tooltips=TOOLTIPS)
  p.circle(log_datas[i]["Error_y"], log_datas[i]["Error_z"], legend_label="y_z", color=palette[4], size=3, alpha=0.4)
  p.legend.location = "top_left"
  p.legend.click_policy = "hide"
  p.background_fill_color="#f0f0f0"
  p.grid.grid_line_color="#f5f5f5"
  plots.append(p)

plot = gridplot(plots, ncols=1, toolbar_location="right", plot_width=1200, plot_height=600)
output_file(folder_path + "/Base-FKBase_yz.html")
show(plot)
#save(plot)



In [47]:
# Err_vector4 => log_datas[i]["Err_vector"][0][3]

#Err_vector4 , Err_vector 5 Graph

Err_vector4 , Err_vector5, Err_vector6= [], [], []
for i in range(0, len(file_list_py)):
  for x in range(0, len(log_datas[i]["Err_vector"])):
    Err_vector4.append(log_datas[i]["Err_vector"][x][3][0])
    Err_vector5.append(log_datas[i]["Err_vector"][x][4][0])
    Err_vector6.append(log_datas[i]["Err_vector"][x][5][0])

plots = []
for i in range(0, len(file_list_py)):
  curdoc().theme = "caliber"

  TOOLTIPS = [
    ("x_value", "$x"),
    ("y_value", "$y)"),
  ]
  p = figure(title = file_list_py[i], x_axis_label="Err_vector4", y_axis_label="Err_vector5", toolbar_location="below", plot_width=1200, plot_height=600, tooltips=TOOLTIPS)
  p.circle(Err_vector4, Err_vector5, legend_label="Err 4_5", color=palette[0])
  p.legend.location = "top_right"
  p.legend.click_policy = "hide"
  p.background_fill_color="#f0f0f0"
  p.grid.grid_line_color="#f5f5f5"
  plots.append(p)

plot = gridplot(plots, ncols=1, toolbar_location="right", plot_width=1200, plot_height=600)
output_file(folder_path + "/Err 4-5.html")
#show(plot)
save(plot)

# Err_vector4, Err_vector6
plots = []
for i in range(0, len(file_list_py)):
  curdoc().theme = "caliber"
  
  TOOLTIPS = [
    ("x_value", "$x"),
    ("y_value", "$y)"),
  ]
  p = figure(title = file_list_py[i], x_axis_label="Err_vector4", y_axis_label="Err_vector6", toolbar_location="below", plot_width=1200, plot_height=600, tooltips=TOOLTIPS)
  p.circle(Err_vector4, Err_vector5, legend_label="Err 4_6", color=palette[2])
  p.legend.location = "top_right"
  p.legend.click_policy = "hide"
  p.background_fill_color="#f0f0f0"
  p.grid.grid_line_color="#f5f5f5"
  plots.append(p)

plot = gridplot(plots, ncols=1, toolbar_location="right", plot_width=1200, plot_height=600)
output_file(folder_path + "/Err 4-6.html")
#show(plot)
save(plot)

# Err_vector5, Err_vector6
plots = []
for i in range(0, len(file_list_py)):
  curdoc().theme = "caliber"
  
  TOOLTIPS = [
    ("x_value", "$x"),
    ("y_value", "$y)"),
  ]
  p = figure(title = file_list_py[i], x_axis_label="Err_vector5", y_axis_label="Err_vector6", toolbar_location="below", plot_width=1200, plot_height=600, tooltips=TOOLTIPS)
  p.circle(Err_vector5, Err_vector6, legend_label="Err 5_6", color=palette[4])
  p.legend.location = "top_right"
  p.legend.click_policy = "hide"
  p.background_fill_color="#f0f0f0"
  p.grid.grid_line_color="#f5f5f5"
  plots.append(p)

plot = gridplot(plots, ncols=1, toolbar_location="right", plot_width=1200, plot_height=600)
output_file(folder_path + "/Err 5-6.html")
show(plot)
#save(plot)


Output hidden; open in https://colab.research.google.com to view.

In [53]:
plots = []
for i in range(0, len(file_list_py)):
  curdoc().theme = "caliber"
  TOOLTIPS = [
    ("x_value", "$x"),
    ("y_value", "$y)"),
  ]
  p = figure(title = file_list_py[i], x_axis_label="index", y_axis_label="norm", toolbar_location="below", plot_width=1200, plot_height=600, tooltips=TOOLTIPS)
  p.circle(log_datas[i]["index"], log_datas[i]["norm"], legend_label="norm", color=palette[1], size=3, alpha=0.4)
  p.legend.location = "top_left"
  p.legend.click_policy = "hide"
  p.background_fill_color="#f0f0f0"
  p.grid.grid_line_color="#f5f5f5"
  plots.append(p)

plot = gridplot(plots, ncols=1, toolbar_location="below", plot_width=1200, plot_height=600)
output_file(folder_path + "/norm.html")
show(plot)
#save(plot)